In [65]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import sys
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
# Recall y precision
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation


print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]
Spark version: 3.1.2


In [2]:
path='./DATA'
file = os.path.join(path, "users_bgs.csv")

In [3]:
df=pd.read_csv(file, usecols=['u_id','bgg_id','Game','category','Your Rating'], low_memory=False)
df.head()

,u_id,Game,Your Rating,bgg_id,category
0,0,1830: Railways & Robber Barons,-1,421,"1021,1034,1011"
1,0,18AL,7,2612,"1021,1120,1034,1011"
2,0,2 de Mayo,6,36522,"1051,1019"
3,0,23,7,103651,1002
4,0,6 nimmt!,7,432,"1002,1098"


In [4]:
#Transform explicit values to implicit values
df['Rating'] = df['Your Rating'].apply(lambda x: 1 if x>0 else 0)

#### Remove users who only have one board game

In [5]:
#Group by 'user_id'
p=df.groupby('u_id').count()

In [6]:
# Visualise users with a single board game
(p['bgg_id']==1).sum()

6

In [7]:
# Remove the user_id with a single board game
bgu=list(p[p['bgg_id']==1].index)

dfm = df[~df.u_id.isin(bgu)]

#### Modified the id of games and users so that they are correlative

In [9]:
# Create a dictionary for games titles and ids

bgg_id=list(dfm['bgg_id'])
game=list(dfm['Game'])

item_dict={}

for key in bgg_id:
    for value in game:
        item_dict[key] = value
        game.remove(value)
        break 

In [10]:
print(len(set(item_dict.keys())))
print(len(set(item_dict.values())))

36535
36535


In [11]:
# Resample
resample_id_item_dict={}
for index, key in enumerate(item_dict.keys()):
    resample_id_item_dict[key]= index
    

In [12]:
resample_item_dict = {resample_id_item_dict[k]:v for k,v in item_dict.items()}
assert(len(set(resample_item_dict.keys())) == len(set(resample_item_dict.values())))

In [13]:
# Create a dictionary for resample users id

u_id=list(dfm['u_id'].unique())
resample_user_dict={}

for index,key in enumerate(u_id):
    resample_user_dict[key] = index

In [14]:
print(len(set(resample_user_dict.keys())))
print(len(set(resample_user_dict.values())))

2844
2844


In [15]:
# Copy and apply the changes
dfm_r=dfm.copy()
dfm_r['item_id']=dfm['bgg_id'].apply(lambda x: resample_id_item_dict[x])
dfm_r['user_id']=dfm['u_id'].apply(lambda x: resample_user_dict[x])
dfm_r.describe()

,u_id,Your Rating,bgg_id,Rating,item_id,user_id
count,838777.000000,838777.000000,838777.000000,838777.000000,838777.000000,838777.000000
mean,1404.570457,3.423040,91310.618300,0.545138,4722.867555,1403.031329
std,813.788285,4.159144,88829.182464,0.497959,6316.864111,812.113149
min,0.000000,-1.000000,1.000000,0.000000,0.000000,0.000000
25%,700.000000,-1.000000,9209.000000,0.000000,680.000000,700.000000
50%,1375.000000,5.000000,55952.000000,1.000000,2185.000000,1374.000000
75%,2087.000000,7.000000,163976.000000,1.000000,6332.000000,2086.000000
max,2851.000000,10.000000,332853.000000,1.000000,36534.000000,2843.000000


In [16]:
#dfm_r.to_csv('DATA/r_users_bgs.csv')
#path='./DATA'
#file = os.path.join(path, "r_users_bgs.csv")
#dfm_r=pd.read_csv(file, low_memory=False)
#dfm_r.head()

#### Format Data for models

In [17]:

dfratings = pd.DataFrame()

dfratings['user_id']=dfm_r['user_id']
dfratings['item_id']=dfm_r['item_id']
dfratings['rating']=dfm_r['Rating']

print(dfratings.shape)
dfratings.head()

(838777, 3)


,user_id,item_id,rating
0,0,0,0
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [18]:
dfbg = dfm_r.groupby(by=['bgg_id','category'], as_index=False).first()
dfbg.drop(['bgg_id','u_id','Your Rating','Rating', 'user_id'],axis=1, inplace=True)

In [19]:
dfgames = pd.DataFrame()

dfgames['item_id']=dfbg['item_id']
dfgames['title']=dfbg['Game']
dfgames['category']=dfbg['category']

print(dfgames.shape)
dfgames.head()

(36535, 3)


,item_id,title,category
0,747,Die Macher,"1021,1026,1001"
1,5896,Dragonmaster,"1002,1010"
2,292,Samurai,"1009,1035"
3,6308,Tal der Könige,1050
4,15364,Mare Mediterraneum,"1015,1008"


## Content Based

In [20]:
# ----- COMENTARIO

In [21]:
dfgames.head()

,item_id,title,category
0,747,Die Macher,"1021,1026,1001"
1,5896,Dragonmaster,"1002,1010"
2,292,Samurai,"1009,1035"
3,6308,Tal der Könige,1050
4,15364,Mare Mediterraneum,"1015,1008"


In [22]:
dfgames['category'] = dfgames['category'].map(lambda x: x.replace(',', ' '))
print(dfgames['category'])

0        1021 1026 1001
1             1002 1010
2             1009 1035
3                  1050
4             1015 1008
              ...      
36530              1002
36531         1002 1030
36532         1002 2481
36533    1009 1028 1113
36534         1021 1013
Name: category, Length: 36535, dtype: object


In [23]:
dfgames.set_index('title', inplace = True)
dfgames.head()

,item_id,category
title,,
Die Macher,747,1021 1026 1001
Dragonmaster,5896,1002 1010
Samurai,292,1009 1035
Tal der Könige,6308,1050
Mare Mediterraneum,15364,1015 1008


In [24]:
print(np.where(dfgames.index=='Tal der Könige')[0])

[3]


In [25]:
count = CountVectorizer()
count_matrix = count.fit_transform(dfgames['category'])

In [26]:
cosine = cosine_similarity(count_matrix, count_matrix)
cosine

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.40824829],
       [0.        , 1.        , 0.        , ..., 0.5       , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.40824829,
        0.        ],
       ...,
       [0.        , 0.5       , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.40824829, ..., 0.        , 1.        ,
        0.        ],
       [0.40824829, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [27]:
#cos= pd.DataFrame(cosine).to_csv("DATA/cb_cosine.csv")

In [28]:
cos= pd.DataFrame(cosine)
cos.head()

,0,1,2,3,4,5,6,7,8,9,...,36525,36526,36527,36528,36529,36530,36531,36532,36533,36534
0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.333333,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.408248
1,0.0,1.0,0.0,0.0,0.0,0.000000,0.5,0.0,0.5,0.408248,...,0.0,0.5,0.707107,0.0,0.0,0.707107,0.5,0.5,0.000000,0.000000
2,0.0,0.0,1.0,0.0,0.0,0.707107,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.408248,0.000000
3,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,1.0,0.000000,0.5,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [29]:
#cos.reset_index()
#cos.head()

In [30]:
def cb_recommendations(name, cosine = cosine):
    
    recommended = []
    indices = pd.Series(dfgames.index)
    idx = indices[indices == name].index[0]
    score = pd.Series(cosine[idx]).sort_values(ascending = False)
    top_10 = list(score.iloc[0:11].index)
    for i in top_10:
        if idx != i:
            recommended.append(list(dfgames.index)[i])
        
    return recommended

In [31]:
cb_recommendations('Azul')

['Rennaissance Chess',
 'Azul: Stained Glass of Sintra',
 'Three Musketeers Game: Chess Variant',
 'Jaleo',
 'Blinq',
 'Spark',
 'Interplay',
 '0·1 (Zero Point One)',
 'Genial Spezial',
 'Color Wheel']

In [32]:
## Collaborative Filtering

In [33]:
#Split random into training and test datasets
train, test = train_test_split(dfratings, test_size = 0.20, random_state = 42) 

In [34]:
print('There are %s users, %s itmes and %s pairs in the train set' \
      %(train.user_id.unique().shape[0], train.item_id.unique().shape[0], train.shape[0]))
train.head()

There are 2844 users, 34067 itmes and 671021 pairs in the train set


,user_id,item_id,rating
32330,123,1060,0
399743,1313,1287,1
763101,2564,5918,0
79614,259,1340,1
719436,2418,727,1


In [35]:
print('There are %s users, %s itmes and %s pairs in the test set' \
      %(test.user_id.unique().shape[0], test.item_id.unique().shape[0], test.shape[0]))
test.head()

There are 2838 users, 20894 itmes and 167756 pairs in the test set


,user_id,item_id,rating
332615,1114,1151,1
489596,1619,2907,1
499690,1659,647,1
698079,2334,1283,1
288796,963,7547,1


In [36]:
### Popularity

In [37]:
# group the train dataset by item and count the number of users
popular = train.groupby('item_id')['user_id'].count()

In [38]:
popular.head()

item_id
0    294
1     74
2    157
3     65
4    798
Name: user_id, dtype: int64

In [39]:
# Sort in descending order
popularsort = popular.sort_values(ascending=False)

In [40]:
popularsort.shape[0]

34067

In [41]:
popularsort.head()

item_id
51     1322
195    1296
70     1262
242    1179
55     1171
Name: user_id, dtype: int64

In [42]:
popularsort.index #id of the Board Game

Int64Index([   51,   195,    70,   242,    55,     5,    44,    15,    93,
              256,
            ...
            27193, 14744, 27180, 27181, 27182, 27183, 27185, 27188, 27190,
            36533],
           dtype='int64', name='item_id', length=34067)

In [43]:
popularbg = np.zeros(shape=(popularsort.shape[0], 3), dtype=object) #MODIFICAR ESTE NOMBRE

for i, ind in enumerate(popularsort.index):
    idx = ind 
    freq = popularsort[idx]  
    title = resample_item_dict[idx]
    popularbg[i] = [idx, title, freq]
    

In [44]:
popularbg[:10,:]

array([[51, 'Carcassonne', 1322],
       [195, 'Love Letter', 1296],
       [70, 'Codenames', 1262],
       [242, 'Pandemic', 1179],
       [55, 'Catan', 1171],
       [5, '7 Wonders', 1160],
       [44, 'Bohnanza', 1155],
       [15, 'Agricola', 1129],
       [93, 'Dominion', 1113],
       [256, 'Power Grid', 1062]], dtype=object)

In [45]:
# Recall
def recall_at_n(N, test, recommended, train=None):
    """
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return the recall
    """
    if train is not None: 
        
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    intersection = len(set(test) & set(rec_true[:N]))
    return intersection / float(np.minimum(N, len(test)))

In [46]:
# get movies in train per user. For this, group by user and get a list of item ids.
trainUsergby = (train.groupby('user_id')['item_id'].apply(list).reset_index())

In [47]:
trainUsergby.head()

,user_id,item_id
0,0,"[276, 308, 192, 300, 100, 6, 391, 388, 287, 10..."
1,1,"[513, 411, 258, 650, 615, 294, 202, 77, 171, 6..."
2,2,"[791, 751, 360, 795, 629, 799, 763, 758, 725, ..."
3,3,"[823, 409, 157, 5, 369, 843, 591, 876, 877, 84..."
4,4,"[375, 903, 941, 939, 227, 957, 232, 947, 148, ..."


In [48]:
testUsergby = (test.groupby('user_id')['item_id'].apply(list).reset_index())
testUsergby.head()

,user_id,item_id
0,0,"[158, 317, 31, 122, 167, 63, 43, 18, 327, 256,..."
1,1,"[548, 661, 578, 276, 611, 544, 454, 472, 689, ..."
2,2,"[163, 768, 157, 329, 346, 802, 781, 797, 455, ..."
3,3,"[870, 155, 824, 432, 599, 888, 829, 156, 868, ..."
4,4,"[893, 479, 325, 944, 959, 951, 361, 917, 922, ..."


In [49]:
# Merge both df
joinedtt = pd.merge(trainUsergby, testUsergby, how='inner', on='user_id', suffixes=('_train', '_test'))
joinedtt.head()

,user_id,item_id_train,item_id_test
0,0,"[276, 308, 192, 300, 100, 6, 391, 388, 287, 10...","[158, 317, 31, 122, 167, 63, 43, 18, 327, 256,..."
1,1,"[513, 411, 258, 650, 615, 294, 202, 77, 171, 6...","[548, 661, 578, 276, 611, 544, 454, 472, 689, ..."
2,2,"[791, 751, 360, 795, 629, 799, 763, 758, 725, ...","[163, 768, 157, 329, 346, 802, 781, 797, 455, ..."
3,3,"[823, 409, 157, 5, 369, 843, 591, 876, 877, 84...","[870, 155, 824, 432, 599, 888, 829, 156, 868, ..."
4,4,"[375, 903, 941, 939, 227, 957, 232, 947, 148, ...","[893, 479, 325, 944, 959, 951, 361, 917, 922, ..."


In [51]:
# Evaluate the model for different number of recommended elements 

list_topN=[1,5,10,20]

for topN in list_topN:
    rec=popularbg[:, 0]
    r_u=joinedtt.apply(lambda l: recall_at_n(N=topN, test=l[2], recommended=rec, train=l[1]), axis=1)
    print('TopN:',topN,',','recall: ',r_u.mean())

TopN: 1 , recall:  0.26955602536997886
TopN: 5 , recall:  0.22798919426826403
TopN: 10 , recall:  0.1964032629730304
TopN: 20 , recall:  0.16919690904544024


In [52]:
#MAP
def apk(N, test, recommended, train=None):
    """
    Computes the average precision at N given recommendations.
    
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return The average precision at N over the test set
    """
    if train is not None: 
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    predicted = rec_true[:N] # top-k predictions
    
    score = 0.0 # This will store the numerator
    num_hits = 0.0 # This will store the sum of rel(i)

    for i,p in enumerate(predicted):
        if p in test and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits/(i+1.0)

    return score / min(len(test), N)

In [54]:
list_topN=[1,5,10,20]

for topN in list_topN:
    predictions = popularbg[:, 0]
    m = joinedtt.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1)
    print('TopN:',topN,',','map: ',m.mean()) #Mejorar impresion

TopN: 1 , map:  0.26955602536997886
TopN: 5 , map:  0.1577572233967583
TopN: 10 , map:  0.11372292258262635
TopN: 20 , map:  0.07998291413267286


In [55]:
## Matrix Co-Ocurrence

In [56]:
moviesPerUser = (train.groupby('user_id')['item_id']
                 .apply(np.array)
                 .to_dict()
                 )

In [57]:
# calculate the number of items in train
n_items = len(resample_item_dict.keys())
n_items

36535

In [58]:
# co-ocurrance matrix will have shape=[n_items,n_items]
coMatrix = np.zeros((n_items, n_items)) # co-occurrence matrix
for user,movies in moviesPerUser.items():
    for m in movies:
        coMatrix[m, movies] += 1

In [59]:
coMatrix

array([[294.,  42.,  35., ...,   0.,   0.,   0.],
       [ 42.,  74.,  10., ...,   0.,   0.,   0.],
       [ 35.,  10., 157., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   1.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [ ]:
# visualize the matrix
plt.matshow(coMatrix, fignum=1000, cmap=plt.cm.binary)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
def co_occurrance_similarity(item_id, coocurrance, ntop=10):
    """
    Returns the top-N most similar items to a given one, based on the coocurrance matrix
    
    :param item_id: id of input item
    :param cooccurrance: 2-dim numpy array with the co-occurance matrix
    :param ntop: number of items to be retrieved
    
    :return top-N most similar items to the given item_id
    """
    similarItems = coocurrance[item_id, :]
    # return indeces of most similar items in descendign order
    mostSimilar = np.argsort(similarItems)[::-1]
    # remove the first element, as it is the item itslef
    mostSimilar = mostSimilar[1:ntop+1]
    
    # return a numpy array with the index (first column) and the value (second column) of the most similar items
    return np.stack((mostSimilar, similarItems[mostSimilar])).T

In [ ]:
def co_occurrance_recommendation(items_id, cooccurrance, ntop=10):
    """
    Obtain the list of ntop recommendations based on a list of items (user history of views)
    
    :param items_id: list of items ids
    :param coocurrence: co-ocurrence matrix (numpy 2-dim array)
    :param ntop: top-K items to be retrieved
    
    :return list of ntop items recommended
    """
    # put together all the similar items and its value. For this, use np.vstack, wich stacks one array after 
    # another (row wise)
    list_sim_items = np.vstack([co_occurrance_similarity(id_, cooccurrance, ntop) for id_ in items_id])
    # Group by id and take the maximum frquency to remove duplicates
    largest_freq = pd.DataFrame(list_sim_items, columns=['id', 'freq']).groupby('id').agg(max).reset_index()
    
    # sort by value in descending order
    sorted_list = largest_freq.sort_values(by='freq', ascending=False)
    
    # get the top N
    out = sorted_list.values[:ntop, 0]
    return out

In [ ]:
# get users in train with their movies
trainUsersGrouped = train.groupby('user_id')['item_id'].apply(list).reset_index()
trainUsersGrouped.head()

In [ ]:
Ntop = 20
# Get the recommendations for all users using the apply method
predictions = trainUsersGrouped.item_id.apply(lambda x: co_occurrance_recommendation(x, coMatrix, Ntop))
predictions.head()

In [ ]:
# get users in test with their movies
testUsersGrouped = test.groupby('user_id')['item_id'].apply(list).reset_index()
testUsersGrouped.head()

In [ ]:
for (seen, recom) in zip(testUsersGrouped.values[:5, 1], predictions[:5]):
    print("*"*6)
    print("Seen items: ")
    print([resample_item_dict[i] for i in seen])
    print("Recommended items: ")
    print([resample_item_dict[i] for i in recom]) #QUITAR ELEMENTOS MOSTRADOS

In [ ]:
topN=30
# add a prediction column to train
trainUsersGrouped['prediction'] = trainUsersGrouped.item_id.apply(
    lambda x: co_occurrance_recommendation(x, coMatrix, topN)
)

In [ ]:
trainUsersGrouped.head()

In [ ]:
# Join the df with train and predictions with the test df
joined = pd.merge(trainUsersGrouped, testUsersGrouped, how='inner', on='user_id', suffixes=('_train', '_test'))

In [ ]:
joined.head()

In [ ]:
def evaluate_recall(topN, trainGrouped, testGrouped, coMatrix, popularity_baseline):
    # add a prediction column to train
    trainUsersGrouped['prediction'] = trainUsersGrouped.item_id.apply(
        lambda x: co_occurrance_recommendation(x, coMatrix, topN)
    )
    # join with test data
    joined = pd.merge(trainUsersGrouped, testUsersGrouped, how='inner', on='user_id', suffixes=('_train', '_test'))
    # calculate average recall
    recall = joined.apply(lambda l: 
                 recall_at_n(N=topN, test=l[3], recommended=l[2], train=l[1]), axis=1).mean()
    print("Co-occurance model: recall@%s=%.3f"%(topN, recall))
    # calculate average recall for the baseline
    recall_baseline = joined.apply(lambda l: 
                 recall_at_n(N=topN, test=l[3], recommended=popularity_baseline, train=l[1]), axis=1).mean()
    print("Popularity model: recall@%s=%.3f"%(topN, recall_baseline))    
    return recall, recall_baseline

In [ ]:
for k in [5,10,20,30]:
    evaluate_recall(k, trainUsersGrouped, testUsersGrouped, coMatrix, popularbg[:, 0]);

In [ ]:
## CF ALS

In [ ]:
#Creamos una sesion de spark
spark = SparkSession.builder.getOrCreate()

In [ ]:
'''El análogo a Pandas dataframe es Spark dataframe, al cual es necesario darle un schema de construcción'''

schema = StructType(
    (
        StructField("user_id", IntegerType()),
        StructField("item_id", IntegerType()),
        StructField("rating", IntegerType()),
    )
)

df_ratings = spark.createDataFrame(dfratings, schema=schema)
df_ratings.show(5)

schema = StructType(
    (
        StructField("item_id", IntegerType()),
        StructField("title", StringType()),
        StructField("category", StringType()),
    )
)


df_games = spark.createDataFrame(dfgames, schema=schema)
df_games.show(5, False)


In [ ]:
#Split the data using the Spark random 
train, test = df_ratings.randomSplit([0.8, 0.2])
print ("N train", train.cache().count())
print ("N test", test.cache().count())

In [ ]:
#Construimos el modelo
'''Configuración del modelo ALS en Spark'''

rec = ALS(maxIter=10, regParam=0.01, implicitPrefs=True, userCol='user_id', itemCol='item_id', ratingCol='rating', nonnegative=True, coldStartStrategy="drop")

'''
Parámetros:

* rank:        Number of latent factors in the model (defaults to 10).
* maxIter:     Maximum number of iterations to run (defaults to 10).
* regParam:    Regularization parameter in ALS (defaults to 1.0).
* nonnegative: Use nonnegative constraints for least squares (defaults to false).

'''

In [ ]:
#Entrenamos el modelo
model = rec.fit(train)

In [ ]:
#Vemos las predicciones
predictions = model.transform(test)
predictions.show(10)

In [ ]:
'''Escogemos un usuario y rankeamos las predicciones'''

user = 19

predictions.filter(F.col('user_id') == user)\
                 .orderBy(F.col('item_id')).show(5)  

predictions.filter(F.col('user_id') == user)\
                 .orderBy(F.col('prediction'), ascending=False).show(5)

In [ ]:
# Evaluacion
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [ ]:
# Contabilizamos el error
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [ ]:
# Con este comando creamos una nueva columna que es la diferencia y-y_hat
# Y lo pasamos a pandas para verlo en un histograma
predicted_ratings = predictions.withColumn('dif', F.col('rating')-F.col('prediction')).toPandas()
predicted_ratings.head()

In [ ]:
#Realizamos una recomendación
'''Spark-ALS nos ofrece también comandos para la generación de recomendaciones'''

# Las 5 mejores peliculas para todos los usuarios
userRecs = model.recommendForAllUsers(5)

# Filtramos para nuestro usuario concreto
userRecs.filter(F.col('user_id')==user).show(1,False)

In [ ]:
predictions_mod=predictions.select(['user_id','item_id','prediction'])

In [ ]:
TOP_K = 10
rank_eval = SparkRankingEvaluation(test, predictions_mod, k = TOP_K, col_user="user_id", col_item="item_id", 
                                    col_rating="rating", col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

In [ ]:
'''Este proceso nos permite encontrar los hiperparámetros que mejor se ajustan a una predicción.
   Este proceso, requiere ejecutar varias repeticiones del mismo modelo, por lo que consume tiempo 
   de computación.'''


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

(trainingRatings, validationRatings) = df_ratings.randomSplit([80.0, 20.0])

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

paramGrid = ParamGridBuilder()\
                .addGrid(rec.rank, [5, 7, 10,20,30])\
                .addGrid(rec.maxIter, [10])\
                .addGrid(rec.regParam, [0.01,0.05, 0.1, 0.5]).build()

crossval = CrossValidator(estimator=rec, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)
cvModel = crossval.fit(trainingRatings)
predictions = cvModel.transform(validationRatings)

print('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.drop())))

In [ ]:
#Original
'''Este proceso nos permite encontrar los hiperparámetros que mejor se ajustan a una predicción.
   Este proceso, requiere ejecutar varias repeticiones del mismo modelo, por lo que consume tiempo 
   de computación.'''


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

(trainingRatings, validationRatings) = df_ratings.randomSplit([80.0, 20.0])

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

paramGrid = ParamGridBuilder()\
                .addGrid(rec.rank, [5, 7, 10,20])\
                .addGrid(rec.maxIter, [20])\
                .addGrid(rec.regParam, [0.05, 0.1, 0.5]).build()

crossval = CrossValidator(estimator=rec, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)
cvModel = crossval.fit(trainingRatings)
predictions = cvModel.transform(validationRatings)

print('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.drop())))

In [ ]:
best_model=cvModel.bestModel

In [ ]:
path='/DATA'
best_model.write().overwrite().save("DATA/cvModel_bm")

In [ ]:
predictions.show()

In [ ]:
predictions = best_model.transform(validationRatings)
predictions_cv_mod=predictions.select(['user_id','item_id','prediction'])

In [ ]:
# Generate top 10 digital music recommendations for each user
userRecs = cvModel.bestModel.recommendForAllUsers(10)
# Generate top 10 user recommendations for each digital music
boardGameRecs = cvModel.bestModel.recommendForAllItems(10)

userRecs.show(5)
boardGameRecs.show(5)

In [ ]:
TOP_K = 10
rank_eval2 = SparkRankingEvaluation(validationRatings, predictionscv_mod, k = TOP_K, col_user="user_id", col_item="item_id", 
                                    col_rating="rating", col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval2.k,
      "MAP:\t%f" % rank_eval2.map_at_k(),
      "Recall@K:\t%f" % rank_eval2.recall_at_k(), sep='\n')

In [ ]:
boardGameRecs_pandas = cvModel.bestModel.recommendForAllItems(10).toPandas()
userRecs_pandas = cvModel.bestModel.recommendForAllUsers(10).toPandas()
boardGameRecs_pandas['recommendations'][0]
userRecs_pandas['recommendations'][3]

In [ ]:
recs=userRecs[0]

In [ ]:
def get_recs_for_user(recs):
    #Recs should be for a specific user
    recs=recs.select("recommendations.item_id","recommendations.ratings")
    items=recs.select('item_id').toPandas().iloc[0,0]
    ratings=recs.select("ratings").toPandas().iloc[0,0]
    rating_matrix=pd.DataFrame(items, columns=['item_id'])
    rating_matrix['ratings']=ratings
    rating_matrix_ps=sqlContext.createDataFrame(rating_matix)
    return rating_matrix_ps

In [83]:
sameModel = cvModel.load("DATA/cvModel")

In [85]:

spark.stop()